In [1]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [2]:
import os
from langsmith_tracker import set_tracking
from langchain_print import stream_response
from multimodal import MultiModal
from tools_news import GoogleNews

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="15.Agent, Agent-Report-With-Image-Generation")

Langsmith 추적이 활성화되었습니다. [프로젝트명: 15.Agent, Agent-Report-With-Image-Generation]


### RAG + Image Generator Agent(보고서 작성)

웹 검색(Web Search), PDF 문서 기반 검색(RAG), 이미지 생성(Image Generation), 파일 관리 도구(File Management) 등을 통해 보고서를 작성하는 에이전트를 구현합니다.

#### 도구 정의

- 웹 검색(Web Search) : Tavily Search
- PDF 문서 기반 검색(RAG) : Retriever
- 이미지 생성(Image Generation) : Dalle-E
- 파일 관리 도구(File Management) : agent_tools.FileManagement
- 보고서 작성(Report Generation) Agent : 위의 도구들을 모두 통합

모든 도구에 대한 API Key는 환경 변수에 정의되어 있습니다.(없으면, 직접 추가. 방법은 15-Agent/01.Tools.ipynb 참고)


In [39]:
# 검색 도구 정의
from langchain_community.tools import TavilySearchResults


search_tool = TavilySearchResults(k=3)

In [40]:
# PDF 문서 기반 검색 도구 정의
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.tools.retriever import create_retriever_tool


# PDF 로드
loader = PyMuPDFLoader("./data/IS-183_AI 위험 유형 및 사례 분석(최종).pdf")
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# 문서 로드 및 분할
documents = loader.load_and_split(text_splitter)
# FAISS 벡터 저장소 생성
vectorstore = FAISS.from_documents(documents, OpenAIEmbeddings())

# retriever 정의
retriever = vectorstore.as_retriever()
document_template = PromptTemplate.from_template(
    "<document><content>{page_content}</content><page>{page}</page><filename>{source}</filename></document>"
)

# 도구 정의
rag_tool = create_retriever_tool(
    retriever,
    "search_pdf",
    description="use this tool to search for information in the PDF file",
    document_prompt=document_template,
)

In [41]:
print(
    rag_tool.invoke({"query": "네이버가 개발한 AI 기술을 소개하는 문서를 찾아주세요."})
)

<document><content>체계를 구축하고 있는 만큼, 한국도 관련 지원 정책을 마련해야 함
∙과기정통부와 TTA는 ‘신뢰할 수 있는 인공지능 개발 안내서’ 8종* 개발, ‘인공지능 시스템 신뢰성 제고를 위한 
요구사항’ 표준 수립, AI 제품 등에 대한 신뢰성 인증 등을 추진하고 있음
* 일반, 의료, 공공사회, 자율주행(2023.7), 일반, 생성 AI 기반 서비스, 스마트 치안, 채용(2024.3) 
47 과학기술정보통신부 (2024.10), “AI안전연구소 설립·운영계획”, 보도자료</content><page>28</page><filename>./data/IS-183_AI 위험 유형 및 사례 분석(최종).pdf</filename></document>

<document><content>∙현재 국내에서는 여러 인공지능 관련 법안에서도 안전성 보장을 위한 규제 위주의 입법안과 기술의 혁신을 통한 
산업 진흥을 중점으로 하는 입법안이 함께 논의 중
4 과학기술정보통신부 (2024.8), (보도자료) 국민 10명 중 6명, 인공지능 기술 이점이 위협보다 크다 
5 TheAIPI (2024.8), AI vs. Public Opinion: Catching you up on the latest from AIPI</content><page>6</page><filename>./data/IS-183_AI 위험 유형 및 사례 분석(최종).pdf</filename></document>

<document><content>ㅇ 엔비디아의 생성 AI 플랫폼 ‘네모’가 소설 저작권을 침해했다고 기소하는 등 데이터 
저작권에 대한 분쟁 확대 (2024.3)
ㅇ 미국 저작권청은 미드저니를 통해 생성한 작품인 ‘새벽의 자리야’는 작품 자체에 대한 
저작권은 인정하지 않고, AI로 만든 이미지와 텍스트를 조정한 것에 대해서만 인정 (2023.9)
43 Nasr 외(2023.11), Scalable Extraction of Training Data from (Production) L

#### 이미지 생성 도구 설명

 Dall-E 이미지 생성 도구를 활용하는 예제입니다.

**주요 속성**
- model: 사용할 DALL-E 모델 이름 (기본값: "dall-e-2", "dall-e-3")
- n: 생성할 이미지 수 (기본값: 1)
- size: 생성할 이미지 크기
  - "dall-e-2": "1024x1024", "512x512", "256x256"
  - "dall-e-3": "1024x1024", "1792x1024", "1024x1792"
- style: 생성될 이미지의 스타일 (기본값: "natural", "vivid")
- quality: 생성될 이미지의 품질 (기본값: "standard", "hd")
- max_retries: 생성 시 최대 재시도 횟수

In [64]:
from langchain.tools import tool
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper

# DALL-E 래퍼 인스턴스 생성
dalle = DallEAPIWrapper(model="dall-e-3", size="1024x1024", quality="standard", n=1)


@tool
def image_tool(prompt: str) -> str:
    """
    Generates an image based on the given text description using DALL-E.

    Args:
        prompt (str): A text description of the image to generate

    Returns:
        str: URL of the generated image
    """
    return dalle.run(prompt)

#### 파일 관리 도구 설명

'08.Agent-File-Management.ipynb' 참고


In [65]:
from langchain_community.agent_toolkits import FileManagementToolkit

# 작업 디렉토리 경로 설정
working_directory = "tmp"

# 파일 관리 도구 생성
file_tools = FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=["write_file", "read_file", "list_directory"],
).get_tools()

file_tools

[WriteFileTool(root_dir='tmp'),
 ReadFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp')]

#### 도구 모음 정의

In [66]:
tools = [search_tool, rag_tool, image_tool] + file_tools

# 도구 목록 출력
tools

[TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))),
 Tool(name='search_pdf', description='use this tool to search for information in the PDF file', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x12d594360>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x135dee0c0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page', 'page_content', 'source'], input_types={}, partial_variables={}, template='<document><content>{page_content}</content><page>{page}</page><filename>{source}</filename></document>'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x12d879da0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x135dee0c0>, s

#### 보고서 작성(Report Generation) Agent 생성

In [68]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain_print import AgentStreamParser

# 채팅 메시지 기록 저장소 정의
store = {}

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공, 도구의 이름과 역할을 입력
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant."
            "You are professional researcher."
            "You are use the search_pdf tool to search for information in the PDF file."
            "You can find further information from the web if you need."
            "You are use the file_management tool to manage files."
            "You are use the dalle tool to generate images based on text descriptions."
            "You are write a report based on the information you found.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# llm 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# agent executor 생성
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=False, handle_parsing_errors=True
)


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# ChatMessageHistory 객체를 생성하여 세션 기록 가져오기
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# 스트림 파서 생성
agent_stream_parser = AgentStreamParser()

pdf 검색을 유도하여 Agent를 실행합니다.

In [59]:
# 에이전트 실행
result = agent_with_chat_history.stream(
    {
        "input": "`생성형 AI` 의 위험성과 관련된 유용한 정보들을 PDF 문서에서 찾아서 bullet point로 정리해 주세요. "
        "한글로 작성해주세요."
        "다음으로는 `report.md` 파일을 새롭게 생성하여 정리한 내용을 저장해주세요. \n\n"
        "#작성방법: \n"
        "1. markdown header 2 크기로 적절한 제목을 작성하세요. \n"
        "2. 발췌한 PDF 문서의 페이지 번호, 파일명을 기입하세요(예시: page 10, filename.pdf). \n"
        "3. 정리된 bullet point를 작성하세요. \n"
        "4. 작성이 완료되면 파일을 `report.md` 에 저장하세요. \n"
        "5. 마지막으로 저장한 `report.md` 파일을 읽어서 출력해 주세요. \n"
    },
    config={"configurable": {"session_id": "sessionid:agent3"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: search_pdf
query: 생성형 AI 위험성
Log: 
Invoking: `search_pdf` with `{'query': '생성형 AI 위험성'}`



[관찰 내용]
Observation: <document><content>SPRi이슈리포트IS-183
AI위험유형및사례분석
2
●생성 AI를 기반으로 높은 성과를 내는 조직들은 상대적으로 더 많은 AI 위험 대응 활동을 수행하는 경향
∙위험 인식 및 완화(68%), 위험 완화 프로세스(44%), 감사·편향 검사·위험 평가를 고려한 모델 설계(43%), 
책임 있는 AI 거버넌스 구축(24%) 등 AI의 안전성 확보 업무를 수행하는 비율이 최대 2배 이상 차이
■국내대중의AI기술의위험성에관한인식은상대적으로높지않은상황으로,정부는AI의
이점을극대화하면서도안전성을제공할수있는정책을마련할필요
●과기정통부의 AI 기술에 대한 대국민 설문조사(2024)4 결과에 따르면, 응답자의 절반 이상은 
AI의 잠재적 위험보다 AI 기술이 지닌 이점이 더 많다고 인식
∙‘AI 기술의 이점이 잠재적 위험보다 많다’고 응답한 비율은 57%이고, ‘규제보다 혁신이 중요’하다고 응답한 
비율은 55%로 다수의 응답자들은 AI 기술의 이점*을 위험보다 더 크게 인식
* 주요 이점으로는 일상생활의 편의성 향상(30.6%), 업무 추진의 효율성 증진(19.6%), 산업현장의 생산성 혁신(16%) 순
∙‘AI 기술의 잠재적 위험이 이점보다 더 많다’고 응답한 비율은 19.1%로 나타났으며, 우려되는 AI 기술의 
잠재적 위험으로는 오작동과 악의적 사용으로 인한 피해를 꼽았음
* 가장 우려되는 AI 기술의 잠재적 위험으로는 ‘설계/오작동 발생으로 인한 피해(18.5%)’, ‘악의적 의도로 AI 활용에 따른 피해
(18.3%)’를 비롯하여 프라이버시 문제와 경제 격차 심화 등 응답
∙응답자들은 AI 발전을 위한 가장 중요한 정부의 정책으로는 AI 법 제정과 윤리기준 마련을 꼽았음
* ‘인

Web 검색 툴 사용을 유도하여 Agent를 실행합니다.

In [60]:
# 에이전트 실행
result = agent_with_chat_history.stream(
    {
        "input": "이번에는 `생성형 AI` 의 위험성과 관련된 유용한 정보들을 웹에서 검색한 결과를 정리해주세요."
        "한글로 작성해주세요."
        "다음으로는 `report.md` 파일을 열어서 기존의 내용을 읽고, 웹 검색하여 찾은 정보를 이전에 작성한 형식에 맞춰 뒷 부분에 추가해 주세요. \n\n"
        "#작성방법: \n"
        "1. markdown header 2 크기로 적절한 제목을 작성하세요. \n"
        "2. 정보의 출처(url)를 기입하세요(예시: 출처: 네이버 지식백과). \n"
        "3. 정리된 웹검색 내용을 작성하세요. \n"
        "4. 작성이 완료되면 파일을 `report.md` 에 저장하세요. \n"
        "5. 마지막으로 저장한 `report.md` 파일을 읽어서 출력해 주세요. \n"
    },
    config={"configurable": {"session_id": "sessionid:agent3"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: tavily_search_results_json
query: 생성형 AI 위험성
Log: 
Invoking: `tavily_search_results_json` with `{'query': '생성형 AI 위험성'}`



[관찰 내용]
Observation: [{'url': 'https://www.hankyung.com/article/202409096811i', 'content': '생성형 AI 위험 커져…안전성 확보 시급 | 한국경제 주니어 생글생글 한경 AI! 유 대표는 "워낙 다양한 시나리오로 AI와 대화할 수 있기 때문에 이제 필터링 방식으로 문제를 잡아내기 어려운 측면이 있다"며 "크게 세 가지로 구분해 생성형 AI 문제의 설명이 가능하다"고 말했다. 그는는 "AI 학습에 활용된 데이터의 유출, AI를 범죄에 활용하는 경우 등의 문제도 발생할 수 있다"고 말했다. 생성형 AI 관련 위험 요소는 10개 이상으로 제시할 수 있다. 유 대표는 "이런 생성형 AI 서비스 등의 문제를 가드레일 시스템으로 대응이 가능하다"고 말했다. 한국경제신문의 프리미엄 스타트업 미디어 플랫폼 긱스(Geeks)가 4일 스타트업 뉴스를 브리핑합니다.태블릿 학습 앱 오르조, 대화로 학생 맞춤형 문제풀이 돕는 ‘오르조 AI 코치’ 출시에듀테크 ... 한국경제TV KED Global 한국경제매거진 마켓인사이트 아르떼 BOOK 한경아카데미 텐아시아 생글생글 주니어 생글생글 TESAT 29초영화제 교육센터 블루밍비트 키즈맘 PR Newswire'}, {'url': 'https://talkai.info/ko/blog/is_ai_dangerous/', 'content': '인공 지능의 위험성. 2018년에는 자율주행 Uber 차량이 보행자를 치어 사망하게 하는 사고가 발생했습니다. ... 또한 창작 활동에 생성형 AI를 활용하면 인간의 창의성과 감정 표현이 억제될 수 있습니다. 또한 AI 시스템과의 과도한 상호작용은 또래 간의 의사소통과'},

제대로된 보고서 작성을 요청하는 예제입니다.

In [61]:
# 보고서 작성 요청
result = agent_with_chat_history.stream(
    {
        "input": "`report.md` 파일을 열어서 안의 내용을 출력하세요. "
        "출력된 내용을 바탕으로, 전문적인 수준의 보고서를 작성하세요. "
        "보고서는 총 3개의 섹션으로 구성되어야 합니다:\n"
        "1. 개요: 보고서 abstract 를 300자 내외로 작성하세요.\n"
        "2. 핵심내용: 보고서의 핵심 내용을 작성하세요. 정리된 표를 markdown 형식으로 작성하여 추가하세요. "
        "3. 최종결론: 보고서의 최종 결론을 작성하세요. 출처(파일명, url 등)을 표시하세요."
        "마지막으로 작성된 결과물을 `report-2.md` 파일에 저장하세요."
    },
    config={"configurable": {"session_id": "sessionid:agent3"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: read_file
file_path: report.md
Log: 
Invoking: `read_file` with `{'file_path': 'report.md'}`



[관찰 내용]
Observation: ## 생성형 AI의 위험성

- **페이지 번호**: 6, 11, 18, 27
- **파일명**: IS-183_AI 위험 유형 및 사례 분석(최종).pdf

- 생성 AI를 기반으로 높은 성과를 내는 조직들은 상대적으로 더 많은 AI 위험 대응 활동을 수행하는 경향이 있음.
- 국내 대중의 AI 기술의 위험성에 대한 인식은 상대적으로 높지 않으며, 정부는 AI의 이점을 극대화하면서도 안전성을 제공할 수 있는 정책을 마련할 필요가 있음.
- AI 기술의 잠재적 위험으로는 오작동과 악의적 사용으로 인한 피해가 있으며, 특히 설계/오작동 발생과 악의적 의도로 AI 활용에 따른 피해가 우려됨.
- AI의 오작동 위험은 비현실적인 기대와 과도한 의존으로 인해 발생할 수 있으며, 전문 분야에서의 AI 오류가 보고되고 있음.
- AI 안전을 위한 사회적, 제도적 논의의 글로벌 협력 필요성이 증가하고 있음.
- 미국 NIST는 생성 AI 기술에 특화된 위험 요인을 분류하고 관리 방안을 제시하고 있으며, 12가지 위험 요인을 정의하고 있음.
- 생성 AI의 위험 요인으로는 CBRN 정보 접근, 오류가 있는 콘텐츠 제작, 데이터 프라이버시 문제, 환경적 영향, 해로운 편향 및 균질화, 정보 무결성 문제 등이 있음.

## 웹 검색 결과

- **출처**: [한국경제](https://www.hankyung.com/article/202409096811i)
  - 생성형 AI의 위험이 커지고 있으며, 안전성 확보가 시급하다는 의견이 있음. AI 학습에 활용된 데이터의 유출 및 범죄에 활용되는 경우 등의 문제도 발생할 수 있음.

- **출처**: [TalkAI](https://talkai.info/ko/blog/is_ai_dang

보고서의 내용을 기반으로 이미지를 생성하는 예제입니다.

In [69]:
# 이미 생성 요청
result = agent_with_chat_history.stream(
    {
        "input": "`report-2.md` 파일을 열어서 안의 내용을 출력하세요. "
        "출력된 내용에 어울리는 이미지를 생성하세요. "
        "생성한 이미지의 url 을 markdown 형식으로 보고서의 가장 상단에 추가하세요. "
        "마지막으로 작성된 결과물을 `report-3.md` 파일에 저장하세요."
    },
    config={"configurable": {"session_id": "sessionid:agent3"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: read_file
file_path: report-2.md
Log: 
Invoking: `read_file` with `{'file_path': 'report-2.md'}`



[관찰 내용]
Observation: # 생성형 AI의 위험성 보고서

## 1. 개요
생성형 AI는 다양한 분야에서 혁신을 가져오고 있지만, 그에 따른 위험성도 증가하고 있다. 본 보고서는 생성형 AI의 위험성을 분석하고, 이를 해결하기 위한 방안을 모색한다. 특히, AI의 오작동, 데이터 프라이버시 문제, 그리고 사회적 영향 등을 다루며, 안전성을 확보하기 위한 정책적 접근이 필요함을 강조한다.

## 2. 핵심내용
| 위험 요인 | 설명 |
|------------|------|
| 오작동 | 비현실적인 기대와 과도한 의존으로 인한 AI의 오류 발생 |
| 악의적 사용 | AI 기술이 범죄에 악용될 가능성 |
| 데이터 유출 | AI 학습에 사용된 데이터의 유출 문제 |
| 프라이버시 문제 | 사용자 정보가 제3자와 공유될 위험 |
| 창의성 억제 | AI의 활용이 인간의 창의성과 감정 표현을 억제할 수 있음 |
| 규제 위반 | AI 도구 사용으로 인한 규제 위반 위험 |

## 3. 최종결론
생성형 AI의 위험성은 단순한 기술적 문제를 넘어 사회적, 윤리적 문제로 확장되고 있다. 따라서, 정부와 기업은 AI의 안전성을 확보하기 위한 정책과 규제를 마련해야 하며, 글로벌 협력이 필요하다. 이러한 노력이 없을 경우, 생성형 AI의 발전은 오히려 사회에 부정적인 영향을 미칠 수 있다.

**출처**: IS-183_AI 위험 유형 및 사례 분석(최종).pdf, [한국경제](https://www.hankyung.com/article/202409096811i), [TalkAI](https://talkai.info/ko/blog/is_ai_dangerous/), [CIO](https://www.cio.com/article/3506259/오류

report-3.md 최종 결과입니다.

![Generative AI Risks](https://oaidalleapiprodscus.blob.core.windows.net/private/org-SD76VRjKZn7BhpAtD6EB27g3/user-3UowxZ06VqNttUO7RMIoAsi7/img-uW59MQYV75v9ASlZqPbZpvhx.png?st=2024-12-05T00%3A42%3A42Z&se=2024-12-05T02%3A42%3A42Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-12-05T00%3A12%3A58Z&ske=2024-12-06T00%3A12%3A58Z&sks=b&skv=2024-08-04&sig=xqtXaRSks8g5pQtIIXre6UKTwrfD/ECEHkxyFeANEKo%3D)

# 생성형 AI의 위험성 보고서

## 1. 개요
생성형 AI는 다양한 분야에서 혁신을 가져오고 있지만, 그에 따른 위험성도 증가하고 있다. 본 보고서는 생성형 AI의 위험성을 분석하고, 이를 해결하기 위한 방안을 모색한다. 특히, AI의 오작동, 데이터 프라이버시 문제, 그리고 사회적 영향 등을 다루며, 안전성을 확보하기 위한 정책적 접근이 필요함을 강조한다.

## 2. 핵심내용
| 위험 요인 | 설명 |
|------------|------|
| 오작동 | 비현실적인 기대와 과도한 의존으로 인한 AI의 오류 발생 |
| 악의적 사용 | AI 기술이 범죄에 악용될 가능성 |
| 데이터 유출 | AI 학습에 사용된 데이터의 유출 문제 |
| 프라이버시 문제 | 사용자 정보가 제3자와 공유될 위험 |
| 창의성 억제 | AI의 활용이 인간의 창의성과 감정 표현을 억제할 수 있음 |
| 규제 위반 | AI 도구 사용으로 인한 규제 위반 위험 |

## 3. 최종결론
생성형 AI의 위험성은 단순한 기술적 문제를 넘어 사회적, 윤리적 문제로 확장되고 있다. 따라서, 정부와 기업은 AI의 안전성을 확보하기 위한 정책과 규제를 마련해야 하며, 글로벌 협력이 필요하다. 이러한 노력이 없을 경우, 생성형 AI의 발전은 오히려 사회에 부정적인 영향을 미칠 수 있다.

**출처**: IS-183_AI 위험 유형 및 사례 분석(최종).pdf, [한국경제](https://www.hankyung.com/article/202409096811i), [TalkAI](https://talkai.info/ko/blog/is_ai_dangerous/), [CIO](https://www.cio.com/article/3506259/오류-출력-및-규제-위반-위험성-높아···-전문가.html)